In [1]:
import pandas as pd

input = pd.read_csv('input.csv')
input.head(10)

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [2]:
unique_labels = input['label'].unique()

unique_labels_list = unique_labels.tolist()

In [3]:
import spacy

nlp = spacy.blank("en")

textcat = nlp.add_pipe("textcat")

In [4]:
for label in unique_labels_list:
    textcat.add_label(label)

In [5]:
# # Add labels to text classifier
# textcat.add_label("ham")
# textcat.add_label("spam")

In [6]:
train_texts = input['text'].values
train_labels = [{'cats': {unique_label: label == unique_label for unique_label in unique_labels_list}} 
                for label in input['label']]

In [7]:
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  {'cats': {'ham': True, 'spam': False}}),
 ('Ok lar... Joking wif u oni...', {'cats': {'ham': True, 'spam': False}}),
 ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  {'cats': {'ham': False, 'spam': True}})]

In [8]:
# from sklearn.linear_model import LogisticRegression
# classifier = LogisticRegression()

In [9]:
# classifier.fit(train_texts,train_labels)

In [10]:
from spacy.util import minibatch
from spacy.training.example import Example

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

batches = minibatch(train_data, size=8)
for batch in batches:
    for text, labels in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, labels)
        nlp.update([example], sgd=optimizer)

In [11]:
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    batches = minibatch(train_data, size=8)
    for batch in batches:   
        for text, labels in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, labels)
            nlp.update([example], sgd=optimizer, losses=losses)
  

In [12]:
nlp.to_disk("/home/outbreakkp/Desktop/txtclass/spacy_model")

In [13]:
# import spacy
# import onnx
# import onnx_tf
# import onnxsim

# nlp = spacy.blank("en")
# textcat = nlp.add_pipe("textcat")
# # Add your label names to the textcat (as you did in your original code)
# # ...

# nlp.from_disk("/home/outbreakkp/Desktop/txtclass/spacy_model")

# onnx_model = onnx_tf.export(nlp, [("text", "text")])  # Specify your input field name as "text"

# onnx.save_model(onnx_model, "nlp_model.onnx")




In [14]:
texts = ["Are you ready for the tea party????? It's gonna be wild",
         "URGENT Reply to this message for GUARANTEED FREE TEA" ]
docs = [nlp.tokenizer(text) for text in texts]
    
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)

print(scores)

[[0.31227258 0.6877274 ]
 [0.74424994 0.25575003]]


In [15]:
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['spam', 'ham']
